In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data
# !tar -xzf ./data/data194484/data_6k.tar.gz
!tar -xzf ./data/data194709/data_full.tar.gz

data194484  data194709
^C


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [3]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

mkdir: 无法创建目录"/home/aistudio/external-libraries": 文件已存在
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 418.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [ ]:
# -*- coding: utf-8 -*-

import os, sys, random, time
import numpy as np
import paddle
import paddle.nn as nn
from paddle.io import DataLoader
import paddle.optimizer as optim
from matplotlib import cm

from Unet_model import UNet2d
from FNO_model import FNO2d
from Trans_model import FourierTransformer2D
import read_data
import utils

######## Settings ########

# 总体迭代步数，原文采用80k，这里采用200k，注意训练的epoch采用iterations进行运算，以避免不同训练集大小优化步数不同
iterations = 200000
# batch size
batch_size = 10
# learning rate, generator
lrG = 0.0005
# decay learning rate?
decayLr = True
# channel exponent to control network size，注意仅用于UNet，对FNO和Transformer不起作用
expo = 7
# data set config  第一个位置为总体样本数量，后三个数表示从reg/sup/shear三个文件夹选择的样本比例，和为1
# prop = None  # by default, use all from "../data/train"
prop = [12800, 1.0, 0, 0.0]
# save txt files with per epoch loss?
saveL1 = True
# model type UNet/FNO/Transformer
net = 'Transformer'
# data path
data_path = os.path.join('data')

##########################

work_path = os.path.join('demo', net, "prop-" + str(prop), "expo-" + str(expo))

prefix = work_path + "/"
utils.makeDirs([prefix])
print("Output prefix: {}".format(prefix))

dropout = 0.  # note, the original runs from https://arxiv.org/abs/1810.08217 used slight dropout, but the effect is minimal; conv layers "shouldn't need" dropout, hence set to 0 here.
doLoad = ""  # optional, path to pre-trained model
print("net: {}".format(net))
print("LR: {}".format(lrG))
print("LR decay: {}".format(decayLr))
print("Iterations: {}".format(iterations))
print("Dropout: {}".format(dropout))

##########################

# seed = random.randint(0, 2 ** 32 - 1)
seed = 2023 
print("Random seed: {}".format(seed))
random.seed(seed)
np.random.seed(seed)
paddle.seed(seed)
# paddle.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic=True # warning, slower

# create pytorch data object with dfp dataset
train_path = os.path.join(data_path, 'train/')
valid_path = os.path.join(data_path, 'test/')
data = read_data.TurbDataset(prop, shuffle=1, dataDir=train_path, dataDirTest=valid_path)
trainLoader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
print("Training batches: {}".format(len(trainLoader)))
dataValidation = read_data.ValiDataset(data)
validLoader = DataLoader(dataValidation, batch_size=batch_size, shuffle=False, drop_last=True)
print("Validation batches: {}".format(len(validLoader)))

# setup training
epochs = int(iterations / len(trainLoader) + 0.5)
if 'UNet' in net:
    net_model = UNet2d(channelExponent=expo, dropout=dropout)
elif 'FNO' in net:
    net_model = FNO2d(in_dim=3, out_dim=3, modes=(32, 32), width=32, depth=4, steps=1, padding=4,
                        activation='gelu')
elif 'Transformer' in net:
    import yaml

    with open(os.path.join('transformer_config.yml')) as f:
        config = yaml.full_load(f)
    config = config['Transformer']
    net_model = FourierTransformer2D(**config)

print(net_model)  # print full net
model_parameters = filter(lambda p: ~p.stop_gradient, net_model.parameters())
params = sum([np.prod(p.shape) for p in model_parameters])
print("Initialized TurbNet with {} trainable params ".format(params))

if len(doLoad) > 0:
    net_model.load_state_dict(paddle.load(doLoad))
    print("Loaded model " + doLoad)

criterionL1 = nn.L1Loss()
optimizerG = optim.Adam(parameters=net_model.parameters(), learning_rate=lrG, beta1=0.5, beta2=0.999,
                        weight_decay=0.0)

##########################

for epoch in range(epochs):

    star_time = time.time()
    # compute LR decay
    if decayLr:
        currLr = utils.computeLR(epoch, epochs, lrG * 0.1, lrG)
        optimizerG.set_lr(currLr)

    logline = "Starting epoch {} / {}, LR {}".format((epoch + 1), epochs, currLr)
    print(logline)

    net_model.train()
    L1_accum = 0.0
    for i, traindata in enumerate(trainLoader, 0):
        inputs, targets = traindata

        optimizerG.clear_grad()
        gen_out = net_model(inputs)

        lossL1 = criterionL1(gen_out, targets)
        lossL1.backward()

        optimizerG.step()

        lossL1viz = lossL1.item()
        L1_accum += lossL1viz

    # validation
    net_model.eval()
    L1val_accum = 0.0
    for i, validata in enumerate(validLoader, 0):
        inputs, targets = validata
        with paddle.no_grad():
            outputs = net_model(inputs)
            lossL1 = criterionL1(outputs, targets)
            L1val_accum += lossL1.item()

    # data for graph plotting
    L1_accum /= len(trainLoader)
    L1val_accum /= len(validLoader)

    logline = "Epoch: {}, batch-idx: {}, L1: {}, L1_val {}, Cost: {}" \
        .format(epoch, i, L1_accum, L1val_accum, time.time() - star_time)
    print(logline)

    if saveL1:
        if epoch == 0:
            utils.resetLog(prefix + "L1tra.txt")
            utils.resetLog(prefix + "L1val.txt")
        utils.log(prefix + "L1tra.txt", "{} ".format(L1_accum), False)
        utils.log(prefix + "L1val.txt", "{} ".format(L1val_accum), False)

    if epoch % 10 == 0:

        for show_id in range(batch_size):
            input_ndarray = inputs.numpy()[show_id]
            v_norm = (np.max(np.abs(input_ndarray[0, :, :])) ** 2 +
                        np.max(np.abs(input_ndarray[1, :, :])) ** 2) ** 0.5

            outputs_denormalized = data.denormalize(outputs.numpy()[show_id], v_norm)
            targets_denormalized = data.denormalize(targets.numpy()[show_id], v_norm)
            utils.makeDirs([prefix + "results_train"])
            utils.imageOut(prefix + "results_train/show-{}".format(show_id), outputs_denormalized,
                            targets_denormalized, saveTargets=True, cmap=cm.RdBu_r)

paddle.save(net_model.state_dict(), prefix + "net_model")

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


Output prefix: demo/Transformer/prop-[12800, 1.0, 0, 0.0]/expo-7/
net: Transformer
LR: 0.0005
LR decay: True
Iterations: 200000
Dropout: 0.0
Random seed: 2023
Number of data loaded (reg, sup, shear): 12800 0 0
Using fixed maxima [4.65, 2.04, 2.37]
Data stats, input  mean 0.189310, max  1.038964;   targets mean 0.272203 , max 1.000000 
Training batches: 1240
Validation batches: 40


W0319 09:26:35.066027   557 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0319 09:26:35.070050   557 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


FourierTransformer2D(
  (dpo): Dropout(p=0.0, axis=None, mode=upscale_in_train)
  (feat_extract): Identity()
  (downscaler): Linear(in_features=5, out_features=64, dtype=float32)
  (upscaler): Identity()
  (encoder_layers): LayerList(
    (0): SimpleTransformerEncoderLayer(
      (attn): SimpleAttention(
        (linears): LayerList(
          (0): Linear(in_features=64, out_features=64, dtype=float32)
          (1): Linear(in_features=64, out_features=64, dtype=float32)
          (2): Linear(in_features=64, out_features=64, dtype=float32)
        )
        (fc): Linear(in_features=72, out_features=64, dtype=float32)
        (dropout): Dropout(p=0.0, axis=None, mode=upscale_in_train)
      )
      (layer_norm1): LayerNorm(normalized_shape=[64], epsilon=1e-05)
      (layer_norm2): LayerNorm(normalized_shape=[64], epsilon=1e-05)
      (ff): FeedForward(
        (lr1): Linear(in_features=64, out_features=128, dtype=float32)
        (activation): ReLU()
        (lr2): Linear(in_features=12

In [6]:
# -*- coding: utf-8 -*-

import os, sys, random, math
import numpy as np
import paddle
import paddle.nn as nn
from paddle.io import DataLoader
import read_data
from read_data import TurbDataset, ValiDataset
from Unet_model import UNet2d
from FNO_model import FNO2d
from Trans_model import FourierTransformer2D
import utils
from utils import log
from matplotlib import cm

dropout = 0.0
prop = [12800, 1.0, 0, 0.0]
net = 'Transformer'
expo = 7

##########################

data_path = os.path.join('data')
train_path = os.path.join(data_path, 'train/')
valid_path = os.path.join(data_path, 'test/')

data = read_data.TurbDataset(prop, shuffle=1, dataDir=train_path, dataDirTest=valid_path)
dataValidation = ValiDataset(data)
validLoader = DataLoader(dataValidation, batch_size=1, shuffle=False, drop_last=True)
print("Validation batches: {}".format(len(validLoader)))

dataset = TurbDataset(prop, mode=TurbDataset.TEST, dataDir=train_path, dataDirTest=valid_path)
# dataset = TurbDataset(None, mode=TurbDataset.TEST, dataDirTest="../data/test/")
testLoader = DataLoader(dataset, batch_size=1, shuffle=False)

if 'UNet' in net:
    net_model = UNet2d(channelExponent=expo, dropout=dropout)
elif 'FNO' in net:
    net_model = FNO2d(in_dim=3, out_dim=3, modes=(32, 32), width=32, depth=4, steps=1, padding=4, activation='gelu')
elif 'Transformer' in net:
    import yaml

    with open(os.path.join('transformer_config.yml')) as f:
        config = yaml.full_load(f)
    config = config['Transformer']
    net_model = FourierTransformer2D(**config)


work_path = os.path.join('demo', net, "prop-" + str(prop), "expo-" + str(expo))

prefix = work_path + "/"
utils.makeDirs([prefix])
print("Output prefix: {}".format(prefix))

lf = os.path.join('demo', net, "prop-" + str(prop), "expo-" + str(expo)) + "/testout.txt"
utils.makeDirs([prefix + "results_test"])
utils.resetLog(lf)

modelFn = prefix + "net_model"

log(lf, "Loading " + modelFn)
net_model.set_state_dict(paddle.load(modelFn))
log(lf, "Loaded " + modelFn)
# netG.cuda()

criterionL1 = nn.L1Loss()
# criterionL1.cuda()
L1val_accum = 0.0
L1val_dn_accum = 0.0
lossPer_p_accum = 0
lossPer_v_accum = 0
lossPer_accum = 0

net_model.eval()

for i, data in enumerate(testLoader, 0):
    inputs, targets = data
    with paddle.no_grad():
        outputs = net_model(inputs)
    outputs = outputs[0]
    targets = targets[0]

    lossL1 = criterionL1(outputs, targets)
    L1val_accum += lossL1.item()

    outputs = np.array(outputs)
    targets = np.array(targets)

    # precentage loss by ratio of means which is same as the ratio of the sum
    lossPer_p = np.sum(np.abs(outputs[0] - targets[0])) / np.sum(np.abs(targets[0]))
    lossPer_v = (np.sum(np.abs(outputs[1] - targets[1])) + np.sum(np.abs(outputs[2] - targets[2]))) \
                / (np.sum(np.abs(targets[1])) + np.sum(np.abs(targets[2])))
    lossPer = np.sum(np.abs(outputs - targets)) / np.sum(np.abs(targets))
    lossPer_p_accum += lossPer_p.item()
    lossPer_v_accum += lossPer_v.item()
    lossPer_accum += lossPer.item()

    log(lf, "Test sample %d" % i)
    log(lf, "    pressure:  abs. difference, ratio: %f , %f " % (np.sum(np.abs(outputs[0] - targets[0])),
                                                                    lossPer_p.item()))
    log(lf, "    velocity:  abs. difference, ratio: %f , %f " % (np.sum(np.abs(outputs[1] - targets[1])) +
                                                                    np.sum(np.abs(outputs[2] - targets[2])),
                                                                    lossPer_v.item()))
    log(lf, "    aggregate: abs. difference, ratio: %f , %f " % (np.sum(np.abs(outputs - targets)),
                                                                    lossPer.item()))

    # Calculate the norm
    input_ndarray = inputs.numpy()[0]
    v_norm = (np.max(np.abs(input_ndarray[0, :, :])) ** 2 + np.max(np.abs(input_ndarray[1, :, :])) ** 2) ** 0.5

    outputs_denormalized = dataset.denormalize(outputs, v_norm)
    targets_denormalized = dataset.denormalize(targets, v_norm)

    # denormalized error
    outputs_denormalized_comp = np.array([outputs_denormalized])
    # outputs_denormalized_comp=torch.from_numpy(outputs_denormalized_comp)
    targets_denormalized_comp = np.array([targets_denormalized])
    # targets_denormalized_comp=torch.from_numpy(targets_denormalized_comp)

    # targets_denormalized_comp, outputs_denormalized_comp = targets_denormalized_comp.float().cuda(), outputs_denormalized_comp.float().cuda()

    # outputs_dn.data.resize_as_(outputs_denormalized_comp).copy_(outputs_denormalized_comp)
    # targets_dn.data.resize_as_(targets_denormalized_comp).copy_(targets_denormalized_comp)

    outputs_dn = paddle.to_tensor(outputs_denormalized_comp)
    targets_dn = paddle.to_tensor(targets_denormalized_comp)

    lossL1_dn = criterionL1(outputs_dn, targets_dn)
    L1val_dn_accum += lossL1_dn.item()

    # write output image, note - this is currently overwritten for multiple models
    utils.imageOut(prefix + "results_test/" + "%04d" % (i), outputs, targets, normalize=False,
                    saveMontage=True, cmap=cm.RdBu_r)  # write normalized with error

log(lf, "\n")
L1val_accum /= len(testLoader)
lossPer_p_accum /= len(testLoader)
lossPer_v_accum /= len(testLoader)
lossPer_accum /= len(testLoader)
L1val_dn_accum /= len(testLoader)
log(lf, "Loss percentage (p, v, combined): %f %%    %f %%    %f %% " %
    (lossPer_p_accum * 100, lossPer_v_accum * 100, lossPer_accum * 100))
log(lf, "L1 error: %f" % (L1val_accum))
log(lf, "Denormalized error: %f" % (L1val_dn_accum))
log(lf, "\n")




Number of data loaded (reg, sup, shear): 12800 0 0
Using fixed maxima [4.65, 2.04, 2.37]
Data stats, input  mean 0.189263, max  1.014262;   targets mean 0.270640 , max 1.000000 
Validation batches: 400
Number of data loaded (reg, sup, shear): 12800 0 0
Using fixed maxima [4.65, 2.04, 2.37]
Data stats, input  mean 0.190220, max  0.961052;   targets mean 0.289502 , max 1.000000 
Output prefix: demo/UNet/prop-[12800, 1.0, 0, 0.0]/expo-7/
Loading demo/UNet/prop-[12800, 1.0, 0, 0.0]/expo-7/net_model
Loaded demo/UNet/prop-[12800, 1.0, 0, 0.0]/expo-7/net_model
Test sample 0
    pressure:  abs. difference, ratio: 7.185699 , 0.061665 
    velocity:  abs. difference, ratio: 33.834564 , 0.003996 
    aggregate: abs. difference, ratio: 41.020264 , 0.004779 
Test sample 1
    pressure:  abs. difference, ratio: 11.782101 , 0.030640 
    velocity:  abs. difference, ratio: 57.174919 , 0.006369 
    aggregate: abs. difference, ratio: 68.957024 , 0.007366 
Test sample 2
    pressure:  abs. difference, r

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 